# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 9:47AM,263302,15,ALIFBA1762RTV65,"Alliance Pharma, Inc.",Released
1,May 23 2023 9:47AM,263302,15,ALIFBA1762TK2N1,"Alliance Pharma, Inc.",Released
2,May 23 2023 9:47AM,263302,15,ALIFBA1762VH257,"Alliance Pharma, Inc.",Released
3,May 23 2023 9:39AM,263463,10,SO94641,"Senseonics, Incorporated",Released
4,May 23 2023 9:36AM,263462,20,7945023,"Exact-Rx, Inc.",Released
5,May 23 2023 9:32AM,263461,10,Eme-113114,Emerginnova,Released
6,May 23 2023 9:32AM,263461,10,Enova-11860,Emerginnova,Released
7,May 23 2023 9:32AM,263461,10,Enova-11861,Emerginnova,Released
8,May 23 2023 9:32AM,263461,10,Enova-11862,Emerginnova,Released
9,May 23 2023 9:32AM,263461,10,Enova-11863,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,263459,Released,4
23,263460,Released,1
24,263461,Released,5
25,263462,Released,1
26,263463,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
263459,NaN,4.0
263460,NaN,1.0
263461,NaN,5.0
263462,NaN,1.0
263463,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
263302,0.0,3.0
263434,18.0,2.0
263439,13.0,10.0
263441,1.0,0.0
263443,4.0,26.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
263302,0,3
263434,18,2
263439,13,10
263441,1,0
263443,4,26


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,263302,0,3
1,263434,18,2
2,263439,13,10
3,263441,1,0
4,263443,4,26


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,263302,,3
1,263434,18,2
2,263439,13,10
3,263441,1,
4,263443,4,26


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc."
3,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated"
4,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc."
5,May 23 2023 9:32AM,263461,10,Emerginnova
10,May 23 2023 9:27AM,263460,10,Ivaoes Animal Health
11,May 23 2023 9:24AM,263459,10,"CLINUVEL, Inc."
15,May 23 2023 9:20AM,263458,10,"Methapharm, Inc."
23,May 23 2023 9:19AM,263457,10,"Methapharm, Inc."
24,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc."
42,May 23 2023 9:02AM,263455,10,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",,3
1,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",,1
2,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",,1
3,May 23 2023 9:32AM,263461,10,Emerginnova,,5
4,May 23 2023 9:27AM,263460,10,Ivaoes Animal Health,,1
5,May 23 2023 9:24AM,263459,10,"CLINUVEL, Inc.",,4
6,May 23 2023 9:20AM,263458,10,"Methapharm, Inc.",,8
7,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",,1
8,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",,18
9,May 23 2023 9:02AM,263455,10,Hush Hush,1,13


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",3,
1,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",1,
2,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",1,
3,May 23 2023 9:32AM,263461,10,Emerginnova,5,
4,May 23 2023 9:27AM,263460,10,Ivaoes Animal Health,1,
5,May 23 2023 9:24AM,263459,10,"CLINUVEL, Inc.",4,
6,May 23 2023 9:20AM,263458,10,"Methapharm, Inc.",8,
7,May 23 2023 9:19AM,263457,10,"Methapharm, Inc.",1,
8,May 23 2023 9:17AM,263456,20,"Exact-Rx, Inc.",18,
9,May 23 2023 9:02AM,263455,10,Hush Hush,13,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",3,
1,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",1,
2,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",1,
3,May 23 2023 9:32AM,263461,10,Emerginnova,5,
4,May 23 2023 9:27AM,263460,10,Ivaoes Animal Health,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",3.0,NaN
1,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",1.0,NaN
2,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",1.0,NaN
3,May 23 2023 9:32AM,263461,10,Emerginnova,5.0,NaN
4,May 23 2023 9:27AM,263460,10,Ivaoes Animal Health,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",3.0,0.0
1,May 23 2023 9:39AM,263463,10,"Senseonics, Incorporated",1.0,0.0
2,May 23 2023 9:36AM,263462,20,"Exact-Rx, Inc.",1.0,0.0
3,May 23 2023 9:32AM,263461,10,Emerginnova,5.0,0.0
4,May 23 2023 9:27AM,263460,10,Ivaoes Animal Health,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2634561,36.0,2.0
15,263302,3.0,0.0
16,263450,0.0,7.0
19,1317227,56.0,44.0
20,1053800,22.0,40.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2634561,36.0,2.0
1,15,263302,3.0,0.0
2,16,263450,0.0,7.0
3,19,1317227,56.0,44.0
4,20,1053800,22.0,40.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,36.0,2.0
1,15,3.0,0.0
2,16,0.0,7.0
3,19,56.0,44.0
4,20,22.0,40.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,15,Released,3.0
2,16,Released,0.0
3,19,Released,56.0
4,20,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,2.0,0.0,7.0,44.0,40.0
Released,36.0,3.0,0.0,56.0,22.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,2.0,0.0,7.0,44.0,40.0
1,Released,36.0,3.0,0.0,56.0,22.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,2.0,0.0,7.0,44.0,40.0
1,Released,36.0,3.0,0.0,56.0,22.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()